# Pre Settings

## PIP

In [1]:
!pip install cloudscraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.2 MB/s eta 0:00:00


## IMPORTS

In [ ]:
import cloudscraper
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
import re
import time
import calendar
import pandas as pd
import json
import http.client

## FUNCTIONS

In [ ]:
def getMonth(month):
  if(month == 'Jan'):
    return 1
  elif(month == "Feb"):
    return 2
  elif(month == "Mar"):
    return 3
  elif(month == "Apr"):
    return 4
  elif(month == "May"):
    return 5
  elif(month == "Jun"):
    return 6
  elif(month == "Jul"):
    return 7
  elif(month == "Aug"):
    return 8
  elif(month == "Sep"):
    return 9
  elif(month == "Oct"):
    return 10
  elif(month == "Nov"):
    return 11
  elif(month == "Dec"):
    return 12
  else:
    return None

def getMonthName(month):
    if month == 1:
        return "Jan"
    elif month == 2:
        return "Feb"
    elif month == 3:
        return "Mar"
    elif month == 4:
        return "Apr"
    elif month == 5:
        return "May"
    elif month == 6:
        return "Jun"
    elif month == 7:
        return "Jul"
    elif month == 8:
        return "Aug"
    elif month == 9:
        return "Sep"
    elif month == 10:
        return "Oct"
    elif month == 11:
        return "Nov"
    elif month == 12:
        return "Dec"
    else:
        return None

In [ ]:
def get24H(day, month, year, am_pm, last = datetime.now()):
  if 'day' in am_pm.lower():
    return datetime(year, month, day, 0, 0)

  addH = 7

  if 'pm' in am_pm:
    pm = am_pm.replace('pm','')
    h = (int(pm.split(':')[0]) + 12)
    if h >= 24:
      h = h%24
      #addH += 24
    m = int(pm.split(':')[1])
  elif 'am' in am_pm:
    am = am_pm.replace('am','')
    h = int(am.split(':')[0])
    m = int(am.split(':')[1])
  else:
    return last

  evnt_date = datetime(year, month, day, h, m) + timedelta(hours=addH)

  return evnt_date

In [ ]:
def saveTable(table):
  endpoint = '/pls/apex/finans_takip/data-transfer/load-day?v_date="Time"&v_curr="Currency"&v_event="Event"&v_forecast="Forecast"&v_actual="Actual"&v_prev="Previous"'

  json_string = ''
  for index, row in table.iterrows():
      url = endpoint
      json_dict = row.to_dict()

      for key in json_dict.keys():
        url = url.replace(f'"{key}"', str(json_dict[key]).replace(' ', '%20'))

      postData(url)


In [ ]:
def getPageHTML(url):       # gets url and returns source html
  scraper = cloudscraper.create_scraper()
  page = scraper.get(url).text
  return page

In [ ]:
from IPython.lib.display import IFrame
def getRecord(url):
  # columns
  c_time = []
  c_curr = []
  c_event = []
  c_forecast = []
  c_actual = []
  c_prev = []

  # page source
  pageHTML = getPageHTML(url)
  soup = BeautifulSoup(pageHTML, 'html.parser')
  table = soup.find('table', class_ = 'calendar__table')
  events = table.find_all('td', class_ = 'calendar__date')

  # start date
  startDate = table.find_next('tr', class_ = 'calendar__row--new-day').find_next('span', class_ = 'date')
  matchObj = re.search('([a-zA-Z]{3})([a-zA-Z]{3}) ([0-9]{1,2})', startDate.text)
  day_of_week = matchObj.group(1)
  month = getMonth(matchObj.group(2))
  day = matchObj.group(3)
  day = int(format(int(day), "02"))
  year = int(url[-4:])

  dt = datetime.now()

  # get event rows
  for event in events:

    # date
    date = event.text.strip()
    if len(date) > 1:
      date = date[3:]
      #print(date)
      month = getMonth(date.split(' ')[0])
      day = date.split(' ')[1]

    # time
    time = event.find_next_sibling('td', class_ = 'time').text.strip()
    dt = get24H(int(day), int(month), int(year), time, dt)
    p_day = format(int(dt.day), "02")
    p_month = format(int(dt.month), "02")
    p_year = format(int(dt.year), "0002")
    hour = format(int(dt.hour), "02")
    minute = format(int(dt.minute), "02")

    # currency
    curr = (event.find_next_sibling('td', class_ = 'currency').text.strip()).replace('\n','').replace(' ', '')
    if len(curr) == 0:
      continue

    # event name
    name = event.find_next_sibling('td', class_ = 'event').find_next('span').text.strip()

    # previous
    previous = (event.find_next_sibling('td', class_ = 'previous').text).replace('\n','').replace(' ', '')
    previous = previous if len(previous) > 1 else 'unknown'

    # forecast
    forecast = (event.find_next_sibling('td', class_ = 'forecast').text).replace('\n','').replace(' ', '')
    forecast = forecast if len(forecast) > 1 else 'unknown'

    # actual
    actual = (event.find_next_sibling('td', class_ = 'actual').text).replace('\n','').replace(' ', '')
    actual = actual if len(actual) > 1 else 'unknown'

    # save row
    if curr == 'USD':
      c_time.append(f'{p_day}/{p_month}/{p_year} {hour}:{minute}')
      c_curr.append(curr)
      c_event.append(name)
      c_forecast.append(forecast)
      c_actual.append(actual)
      c_prev.append(previous)
    else:
      continue


  # create table
  data = {
      'Time': c_time,
      'Currency': c_curr,
      'Event': c_event,
      'Forecast': c_forecast,
      'Actual': c_actual,
      'Previous': c_prev
  }

  data_table = pd.DataFrame(data)

  return data_table

In [ ]:
def postData(endpoint):
  conn = http.client.HTTPSConnection("apex.oracle.com")
  conn.request("POST", endpoint)
  res = conn.getresponse()
  data = res.read()
  return data.decode("utf-8")

# MAIN

In [ ]:
year = 2016
timeline = 'day'

for month in range(1,13):
  monthName = getMonthName(month).lower()
  for day in range(31):
    date = f'{monthName}{day}.{year}'
    url = f'https://www.forexfactory.com/calendar?{timeline}={date}'
    table = getRecord(url)
    saveTable(table)
    display(table)

,Time,Currency,Event,Forecast,Actual,Previous
0,31/12/2016 15:30,USD,Unemployment Claims,274K,287K,267K
1,31/12/2016 16:45,USD,Chicago PMI,50.4,42.9,48.7
2,31/12/2016 17:30,USD,Natural Gas Storage,-54B,-58B,-32B


,Time,Currency,Event,Forecast,Actual,Previous
0,01/01/2016 00:00,USD,Bank Holiday,unknown,unknown,unknown


,Time,Currency,Event,Forecast,Actual,Previous


,Time,Currency,Event,Forecast,Actual,Previous
0,03/01/2016 20:15,USD,FOMC Member Fischer Speaks,unknown,unknown,unknown
1,04/01/2016 00:30,USD,FOMC Member Mester Speaks,unknown,unknown,unknown


,Time,Currency,Event,Forecast,Actual,Previous
0,04/01/2016 16:45,USD,Final Manufacturing PMI,51.1,51.2,51.3
1,04/01/2016 17:00,USD,ISM Manufacturing PMI,49.1,48.2,48.6
2,04/01/2016 17:00,USD,Construction Spending m/m,0.7%,-0.4%,0.3%
3,04/01/2016 17:00,USD,ISM Manufacturing Prices,36.5,33.5,35.5


...

,Time,Currency,Event,Forecast,Actual,Previous


,Time,Currency,Event,Forecast,Actual,Previous


,Time,Currency,Event,Forecast,Actual,Previous
0,11/01/2016 16:59,USD,Labor Market Conditions Index m/m,unknown,2.9,2.7


,Time,Currency,Event,Forecast,Actual,Previous
0,12/01/2016 12:30,USD,FOMC Member Fischer Speaks,unknown,unknown,unknown
1,12/01/2016 13:00,USD,NFIB Small Business Index,95.4,95.2,94.8
2,12/01/2016 17:00,USD,JOLTS Job Openings,5.41M,5.43M,5.35M
3,12/01/2016 17:00,USD,IBD/TIPP Economic Optimism,47.6,47.3,47.2
4,13/01/2016 04:00,USD,President Obama Speaks,unknown,unknown,unknown


,Time,Currency,Event,Forecast,Actual,Previous
0,13/01/2016 17:30,USD,Crude Oil Inventories,1.9M,0.2M,-5.1M
1,13/01/2016 20:03,USD,10-y Bond Auction,unknown,2.09|2.8,2.23|2.6
2,13/01/2016 21:00,USD,Beige Book,unknown,unknown,unknown
3,13/01/2016 21:00,USD,Federal Budget Balance,-2.7B,-14.4B,-64.6B


In [ ]:
import requests
from datetime import datetime, timedelta

def get_historical_usd_btc_prices(start_date, end_date):
   url = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart'
   params = {
       'vs_currency': 'usd',
       'from': start_date.timestamp(),
       'to': end_date.timestamp(),
       'days': (end_date - start_date).days + 1,
       'interval': 'daily'
   }
   response = requests.get(url, params=params)
   data = response.json()
   prices = data
   return prices

# Tarihsel fiyatları almak için işlevi çağırın
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 6, 25)
btc_prices = get_historical_usd_btc_prices(start_date, end_date)
print(btc_prices)


{'prices': [[1672531200000, 16540.693624776803], [1672617600000, 16615.969318012816], [1672704000000, 16674.342536301097], [1672790400000, 16677.646185850346], [1672876800000, 16855.1747387671], [1672963200000, 16832.098988497855], [1673049600000, 16960.23206247515], [1673136000000, 16950.861521853905], [1673222400000, 17074.61539572674], [1673308800000, 17194.90932945318], [1673395200000, 17436.90232978116], [1673481600000, 17996.832553741606], [1673568000000, 18866.810330617045], [1673654400000, 19941.780543296303], [1673740800000, 21019.20661402266], [1673827200000, 20853.230569490253], [1673913600000, 21175.33773662521], [1674000000000, 21156.78393116713], [1674086400000, 20726.844969426445], [1674172800000, 21081.671022204326], [1674259200000, 22705.83367889906], [1674345600000, 22771.023287021784], [1674432000000, 22736.66142892971], [1674518400000, 22984.87497643721], [1674604800000, 22610.010598272846], [1674691200000, 23180.422697874263], [1674777600000, 23024.74618081709], [1

# TEST

In [ ]:
today = datetime.now()

month = getMonthName(today.month).lower()
day = today.day
year = today.year
date = f'{month}{day}.{year}'

timeline = 'day' # day or week

url = f'https://www.forexfactory.com/calendar?{timeline}={date}'

table = getRecord(url)

In [ ]:
table

,Time,Currency,Event,Forecast,Actual,Previous
0,26/05/2023 09:00,GBP,Retail Sales m/m,0.3%,unknown,-0.9%
1,26/05/2023 15:30,USD,Core PCE Price Index m/m,0.3%,unknown,0.3%
2,26/05/2023 15:30,USD,Core Durable Goods Orders m/m,0.0%,unknown,0.2%
3,26/05/2023 15:30,USD,Durable Goods Orders m/m,-1.0%,unknown,3.2%
4,26/05/2023 15:30,USD,Goods Trade Balance,-85.6B,unknown,-84.6B
5,26/05/2023 15:30,USD,Personal Income m/m,0.4%,unknown,0.3%
6,26/05/2023 15:30,USD,Personal Spending m/m,0.4%,unknown,0.0%
7,26/05/2023 15:30,USD,Prelim Wholesale Inventories m/m,0.2%,unknown,0.0%
8,26/05/2023 17:00,USD,Revised UoM Consumer Sentiment,58.2,unknown,57.7
9,26/05/2023 17:00,USD,Revised UoM Inflation Expectations,unknown,unknown,4.5%


In [ ]:
json_string = ''
for index, row in table.iterrows():
    json_string = str(row.to_dict())


In [ ]:
table.to_csv(f'{day}-{month}-{year}_{timeline}.csv', index=False)